In [ ]:
import os
import cv2
import numpy as np
import face_alignment
import pandas as pd
from tqdm import tqdm
from typing import List, Tuple, Optional

def load_images_and_labels(base_dir, emotions):
    """Load images and labels from the specified directory."""
    images = []
    labels = []
    image_names = []

    for emotion in emotions:
        emotion_dir = os.path.join(base_dir, emotion)
        if not os.path.isdir(emotion_dir):
            print(f"Warning: Directory {emotion_dir} does not exist. Skipping...")
            continue
        
        for file_name in os.listdir(emotion_dir):
            file_path = os.path.join(emotion_dir, file_name)
            image = cv2.imread(file_path)
            if image is not None:
                images.append(image)
                labels.append(emotion)
                image_names.append(file_name)
            else:
                print(f"Skipped {file_name}: Unable to read image.")

    return images, labels, image_names

def detect_landmarks(images):
    """Detect facial landmarks in the given images."""
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType.TWO_D, flip_input=False)
    landmarks = []

    for image in tqdm(images, desc="Detecting Landmarks"):
        try:
            preds = fa.get_landmarks(image)
            if preds is not None and len(preds) > 0:
                landmarks.append(preds[0])  # Use the first detected face
            else:
                landmarks.append(None)  # No face detected
                print("No face detected in an image.")
        except Exception as e:
            landmarks.append(None)
            print(f"Error processing image: {e}")

    return landmarks

def save_to_csv(filename, image_names, landmarks, labels):
    """Save image names, landmarks, and labels to a CSV file."""
    data = {
        'Image_Name': image_names,
        'Landmarks': [],
        'Labels': labels
    }

    for landmark in landmarks:
        if landmark is not None:
            # Convert landmarks to the required nested list format and then to string
            nested_landmarks = [[int(point[0]), int(point[1])] for point in landmark]
            data['Landmarks'].append(str(nested_landmarks))  # Save as string representation of nested list
        else:
            data['Landmarks'].append("None")  # If no landmarks detected

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

def tifdf():
    emotions = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]
    
    train_images, train_labels, train_image_names = load_images_and_labels('ChildNet/Train', emotions)
    test_images, test_labels, test_image_names = load_images_and_labels('ChildNet/Test', emotions)
    
    train_landmarks = detect_landmarks(train_images)
    test_landmarks = detect_landmarks(test_images)

    # Save to CSV with checks for file existence
    if not os.path.exists('child_train.csv'):
        save_to_csv('child_train.csv', train_image_names, train_landmarks, train_labels)
        print("child_train.csv created successfully!")
    else:
        save_to_csv('child_train.csv', train_image_names, train_landmarks, train_labels)
        print("child_train.csv already exists.")

    if not os.path.exists('child_test.csv'):
        save_to_csv('child_test.csv', train_image_names, train_landmarks, train_labels)
        print("child_test.csv created successfully!")
    else:
        save_to_csv('child_test.csv', train_image_names, train_landmarks, train_labels)
        print("child_test.csv already exists.")

# Run the function to process and save the data
tifdf()

In [ ]:
import pandas as pd 
df = pd.read_csv('tif_df_train.csv')
df.head()

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import ast 

%matplotlib inline  

# Define paths
train_folder = 'TIF_DF/Train' 
csv_file = 'tif_df_train.csv'  

# Read CSV file
df = pd.read_csv(csv_file)

def show_image_with_landmarks(image_name, landmarks, emotion_label):
    emotion_folder = os.path.join(train_folder, emotion_label)
    image_path = os.path.join(emotion_folder, image_name)
    
    if not os.path.exists(image_path):
        print(f"Image {image_name} not found at {image_path}. Skipping.")
        return
    
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to read image {image_name}. Skipping.")
        return
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)

    landmark_ls = ast.literal_eval(landmarks) 
    for x, y in landmark_ls:
        plt.scatter(x, y, c='red', s=50, marker='x')  
    
    plt.title(f"Image: {image_name}, Emotion: {emotion_label}")  
    plt.axis('off')  # Turn off axis
    plt.show()


for index, row in df.iterrows():
    # if index >= 10:  
    #     break
    
    image_name = row['Image_Name'] 
    emotion_label = row['Labels']  
    
    emotion_folder = os.path.join(train_folder, emotion_label)
    if not os.path.isdir(emotion_folder):
        print(f"Emotion folder {emotion_folder} does not exist. Skipping {emotion_label}.")
        continue
    
    if image_name not in os.listdir(emotion_folder):
        print(f"Image {image_name} not found in {emotion_folder}. Skipping.")
        continue
    
    landmarks = row['Landmarks']  
    show_image_with_landmarks(image_name, landmarks, emotion_label)

In [ ]:
def save_to_csv(filename, image_names, landmarks, labels):
    """Save image names, landmarks, and labels to a CSV file."""
    data = {
        'Image_Name': image_names,
        'Landmarks': [],
        'Labels': labels
    }

    for landmark in landmarks:
        if landmark is not None:
            # Convert landmarks to the required nested list format and then to string
            nested_landmarks = [[int(point[0]), int(point[1])] for point in landmark]
            data['Landmarks'].append(str(nested_landmarks))  # Save as string representation of nested list
        else:
            data['Landmarks'].append("None")  # If no landmarks detected

    df = pd.DataFrame(data)
    df.to_csv(filename, index=False)

def tifdf():
    emotions = ["Angry", "Disgust", "Fear", "Happiness", "Neutral", "Sad", "Surprised"]
    
    # train_images, train_labels, train_image_names = load_images_and_labels('TIF_DF/Train', emotions)
    test_images, test_labels, test_image_names = load_images_and_labels('TIF_DF/Test', emotions)
    
    test_landmarks = detect_landmarks(test_images)
    # test_landmarks = detect_landmarks(test_images)

    # Save to CSV with checks for file existence
    if not os.path.exists('tif_df_test.csv'):
        save_to_csv('tif_df_test.csv', test_image_names, test_landmarks, test_labels)
        print("tif_df_test.csv created successfully!")
    else:
        save_to_csv('tif_df_test.csv', test_image_names, test_landmarks, test_labels)
        print("tif_df_test.csv already exists.")

# Run the function to process and save the data
tifdf()

In [ ]:
import pandas as pd 
df = pd.read_csv('tif_df_test.csv')
df

In [ ]:
import os
import cv2
import pandas as pd
import matplotlib.pyplot as plt
import ast 

%matplotlib inline  

# Define paths
train_folder = 'TIF_DF/Test' 
csv_file = 'tif_df_test.csv'  

# Read CSV file
df = pd.read_csv(csv_file)

def show_image_with_landmarks(image_name, landmarks, emotion_label):
    emotion_folder = os.path.join(train_folder, emotion_label)
    image_path = os.path.join(emotion_folder, image_name)
    
    if not os.path.exists(image_path):
        print(f"Image {image_name} not found at {image_path}. Skipping.")
        return
    
    img = cv2.imread(image_path)
    
    if img is None:
        print(f"Failed to read image {image_name}. Skipping.")
        return
    
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    plt.figure(figsize=(8, 8))
    plt.imshow(img)

    landmark_ls = ast.literal_eval(landmarks) 
    for x, y in landmark_ls:
        plt.scatter(x, y, c='red', s=50, marker='x')  
    
    plt.title(f"Image: {image_name}, Emotion: {emotion_label}")  
    plt.axis('off')  # Turn off axis
    plt.show()


for index, row in df.iterrows():
    # if index >= 10:  
    #     break
    
    image_name = row['Image_Name'] 
    emotion_label = row['Labels']  
    
    emotion_folder = os.path.join(train_folder, emotion_label)
    if not os.path.isdir(emotion_folder):
        print(f"Emotion folder {emotion_folder} does not exist. Skipping {emotion_label}.")
        continue
    
    if image_name not in os.listdir(emotion_folder):
        print(f"Image {image_name} not found in {emotion_folder}. Skipping.")
        continue
    
    landmarks = row['Landmarks']  
    show_image_with_landmarks(image_name, landmarks, emotion_label)